In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import tensorflow_hub as hub
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split

In [5]:
df=pd.read_csv('california_restaurants.csv')

In [6]:
df.columns

Index(['Unnamed: 0', 'business_id', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours', 'user_id',
       'review_stars', 'text', 'date'],
      dtype='object')

In [19]:
pdf=pd.read_csv('preprocessed.csv')

In [20]:
pdf

,Unnamed: 0,text,review_stars,tokens,lemma,stems
0,0,What a great addition to the Funk Zone! Grab ...,5,"['What', 'a', 'great', 'addition', 'to', 'the'...",What a great addition to the Funk Zone Grab a ...,what a great addit to the funk zone grab a bit...
1,1,Possibly the best breakfast sandwich EVER. On...,5,"['Possibly', 'the', 'best', 'breakfast', 'sand...",Possibly the best breakfast sandwich EVER On a...,possibl the best breakfast sandwich ever on a ...
2,2,Pricey ( a ham and cheese croissant was $5.50)...,4,"['Pricey', 'a', 'ham', 'and', 'cheese', 'crois...",Pricey a ham and cheese croissant wa 5 50 a wi...,pricey a ham and chees croissant wa 5 50 as wi...
3,3,Are you kidding me? This is without a doubt so...,5,"['Are', 'you', 'kidding', 'me', 'This', 'is', ...",Are you kidding me This is without a doubt som...,are you kid me thi is without a doubt some of ...
4,4,"Super slow service, ""salads"" are not very good...",2,"['Super', 'slow', 'service', 'salads', 'are', ...",Super slow service salad are not very good and...,super slow servic salad are not veri good and ...
...,...,...,...,...,...,...
205459,205459,This is one of my new favorite pizza joints he...,5,"['This', 'is', 'one', 'of', 'my', 'new', 'favo...",This is one of my new favorite pizza joint her...,thi is one of my new favorit pizza joint here ...
205460,205460,Ordered vegan chicken and Daiya cheese and onl...,4,"['Ordered', 'vegan', 'chicken', 'and', 'Daiya'...",Ordered vegan chicken and Daiya cheese and onl...,order vegan chicken and daiya chees and onli w...
205461,205461,This was the worst pizza I've ever had ever ea...,1,"['This', 'was', 'the', 'worst', 'pizza', 'I', ...",This wa the worst pizza I ve ever had ever eat...,thi wa the worst pizza i ve ever had ever eate...
205462,205462,One of the best pizzas in SB....seriously!!!\n...,5,"['One', 'of', 'the', 'best', 'pizzas', 'in', '...",One of the best pizza in SB seriously This is ...,one of the best pizza in sb serious thi is a r...


In [18]:
df

,text,review_stars
0,What a great addition to the Funk Zone! Grab ...,5
1,Possibly the best breakfast sandwich EVER. On...,5
2,Pricey ( a ham and cheese croissant was $5.50)...,4
3,Are you kidding me? This is without a doubt so...,5
4,"Super slow service, ""salads"" are not very good...",2
...,...,...
205459,This is one of my new favorite pizza joints he...,5
205460,Ordered vegan chicken and Daiya cheese and onl...,4
205461,This was the worst pizza I've ever had ever ea...,1
205462,One of the best pizzas in SB....seriously!!!\n...,5


In [8]:
df=df[['text','review_stars']]

In [21]:
ldf=pdf[['lemma','review_stars']]

In [12]:
from sklearn.utils import class_weight
class_weights = list(class_weight.compute_class_weight(class_weight='balanced',
                                classes=np.unique(df['review_stars']),
                                y =df['review_stars']))

In [57]:
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [58]:
X_train


,text,review_stars
1025,"Location is central, right on the pier with a ...",3
133860,Whenever I am in town and crave some Chinese f...,5
75707,"Great beer, food was alright mostly salty food...",4
102520,Where do I start?\nI can eat just about anythi...,1
93049,So the food and the service was pretty good. I...,2
...,...,...
119879,"Despite a little mixup on arrival, we ended up...",4
103694,Best food ever.. it was about time our town go...,5
131932,Fun place.. hospitality and drinks are incredi...,4
146867,Two of my friends ordered the trout dinner ..T...,1


In [59]:
X_train['review_stars']=X_train["review_stars"]-1

In [60]:
X_test['review_stars']=X_test["review_stars"]-1

In [61]:
class_weights.sort()

In [62]:
df['review_stars'].value_counts()

5    102470
4     45823
3     21149
1     20272
2     15750
Name: review_stars, dtype: int64

In [63]:
class_weights

[0.40102273836244756,
 0.8967723632237086,
 1.9430138540829354,
 2.02707182320442,
 2.6090666666666666]

In [16]:
weights={}
for index,weight in enumerate(class_weights):
    weights[index]=weight

In [17]:
weights

{0: 0.40102273836244756,
 1: 0.8967723632237086,
 2: 1.9430138540829354,
 3: 2.02707182320442,
 4: 2.6090666666666666}

In [64]:
dataset_train=tf.data.Dataset.from_tensor_slices((X_train["text"].values,X_train["review_stars"].values))
dataset_test=tf.data.Dataset.from_tensor_slices((X_test["text"].values,X_test["review_stars"].values))
print("Len of training data",len(dataset_train))
print("Len of testing data",len(dataset_test))

Len of training data 164371
Len of testing data 41093


In [65]:
for text,target in dataset_train.take(1):
    print('Review: {},\n Star: {}'.format(text,target))

Review: b'Location is central, right on the pier with a nice view. Restaurant is small, seats are tight together. Service was okay! We liked our Clam chowder on bread, the rest we ordered was just so so. There are so many other choices just by the pier, we will try other restaurants when we visit the area again next time.',
 Star: 2


In [66]:
for text,target in dataset_test.take(1):
    print('Review: {},\n Star: {}'.format(text,target))

Review: b"Came here with a bachelorette party for lunch. This was a recommendation from a former local I know. \n\nIt's an order at the register and sit down and it's delivered to your table kind of place. We came in around 1 and it was pretty busy. Despite this, the staff set up a table for us right away.\n\nI ordered Baja tacos. I really liked them, they had a little bit too much sauce on them, but overall I would try them again. The other girls ordered, tacos del mar, steak tacos, open face tacos etc. Everyone was really pleasantly surprised and excited. \n\nThe only down side was finding parking.",
 Star: 4


In [67]:
def fetch(text, labels):
    return text,tf.one_hot(labels,5)

In [68]:
train_data_one=dataset_train.map(fetch)
test_data_one=dataset_test.map(fetch)

In [69]:
next(iter(train_data_one))

(<tf.Tensor: shape=(), dtype=string, numpy=b'Location is central, right on the pier with a nice view. Restaurant is small, seats are tight together. Service was okay! We liked our Clam chowder on bread, the rest we ordered was just so so. There are so many other choices just by the pier, we will try other restaurants when we visit the area again next time.'>,
 <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 1., 0., 0.], dtype=float32)>)

In [70]:
next(iter(test_data_one))

(<tf.Tensor: shape=(), dtype=string, numpy=b"Came here with a bachelorette party for lunch. This was a recommendation from a former local I know. \n\nIt's an order at the register and sit down and it's delivered to your table kind of place. We came in around 1 and it was pretty busy. Despite this, the staff set up a table for us right away.\n\nI ordered Baja tacos. I really liked them, they had a little bit too much sauce on them, but overall I would try them again. The other girls ordered, tacos del mar, steak tacos, open face tacos etc. Everyone was really pleasantly surprised and excited. \n\nThe only down side was finding parking.">,
 <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 0., 0., 1.], dtype=float32)>)

In [71]:
embedding="https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"

In [73]:
train_data,train_labels=next(iter(train_data_one.batch(5)))

In [72]:
hub_layer=hub.KerasLayer(embedding,output_shape=[128],input_shape=[],
                        dtype=tf.string,trainable=True)

In [74]:
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 0.75783455,  0.00168396,  0.16151853,  0.30778477,  0.07091644,
         0.13403703, -0.03691614, -0.20054229,  0.04383904,  0.14700083,
        -0.041282  , -0.05731475,  0.0548557 , -0.10773784,  0.15131384,
        -0.02645872, -0.12017765, -0.1300311 , -0.20037632,  0.4689916 ,
         0.01125909,  0.02216557, -0.08325087, -0.02962836,  0.09385838,
        -0.3486678 ,  0.11687928,  0.07690086, -0.06866097,  0.06200006,
        -0.03414959, -0.07064124,  0.24115461,  0.14443798,  0.15180892,
         0.07984551,  0.14023668, -0.13683932, -0.00674952,  0.19818462,
         0.1196639 , -0.03489685, -0.13811593, -0.06081371, -0.02123419,
         0.1591296 , -0.05187413, -0.13239482, -0.36144474, -0.01035706,
         0.04011806,  0.10807533,  0.06838969,  0.03281809, -0.11109102,
         0.04239985, -0.04068753, -0.17173074,  0.03453194,  0.07049307,
        -0.11636937,  0.08905765, -0.04003426, -0.1285472 , -0.09458155,
 

In [75]:
model=tf.keras.Sequential()
model.add(hub_layer)
for units in [128,128,64,32]:
    model.add(tf.keras.layers.Dense(units,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [76]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

In [77]:
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [78]:
train_data_one=train_data_one.shuffle(50000).batch(512)
test_data_one=test_data_one.batch(512)

In [79]:
history=model.fit(train_data_one,
                    epochs=10,
                        validation_data=test_data_one,
                        class_weight=weights)

Epoch 1/10
322/322 [==============================] - 18s 50ms/step - loss: 1.6189 - accuracy: 0.5754 - val_loss: 0.7995 - val_accuracy: 0.6662
Epoch 2/10
322/322 [==============================] - 16s 49ms/step - loss: 1.2949 - accuracy: 0.6715 - val_loss: 0.8038 - val_accuracy: 0.6632
Epoch 3/10
322/322 [==============================] - 16s 49ms/step - loss: 1.1852 - accuracy: 0.7035 - val_loss: 0.8030 - val_accuracy: 0.6636
Epoch 4/10
322/322 [==============================] - 16s 49ms/step - loss: 1.0792 - accuracy: 0.7314 - val_loss: 0.8463 - val_accuracy: 0.6664
Epoch 5/10
322/322 [==============================] - 16s 49ms/step - loss: 0.9531 - accuracy: 0.7624 - val_loss: 0.9120 - val_accuracy: 0.6555
Epoch 6/10
322/322 [==============================] - 16s 49ms/step - loss: 0.8102 - accuracy: 0.7951 - val_loss: 0.9816 - val_accuracy: 0.6567
Epoch 7/10
322/322 [==============================] - 16s 49ms/step - loss: 0.6679 - accuracy: 0.8292 - val_loss: 1.1839 - val_accuracy:

In [81]:
results=model.evaluate(dataset_test.map(fetch).batch(20000),verbose=2)

3/3 - 1s - loss: 1.5290 - accuracy: 0.6336 - 834ms/epoch - 278ms/step


In [82]:
test_data,test_labels=next(iter(dataset_test.map(fetch).batch(30000)))

In [84]:
y_pred=model.predict(test_data)

938/938 [==============================] - 3s 3ms/step


In [85]:
from sklearn.metrics import confusion_matrix,classification_report

In [87]:
print(classification_report(test_labels.numpy().argmax(axis=1),y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.77      0.44      0.56      2884
           1       0.35      0.39      0.37      2250
           2       0.40      0.45      0.43      3065
           3       0.47      0.44      0.45      6805
           4       0.77      0.83      0.80     14996

    accuracy                           0.63     30000
   macro avg       0.55      0.51      0.52     30000
weighted avg       0.63      0.63      0.63     30000

